# Домашнее задание 4.

## Часть 1. Парсинг телеграм-каналов

У вас есть 2 файла на выбор в [этой папке](https://disk.yandex.ru/d/druM43Rd5bdiQg):
- Канал Системный Блокъ
- Канал Нерусский мир

В них сохранена история каналов за 2024 год в html-формате

Подготовьте таблицу, в которой будет:
- отправитель (у нас это будет канал - но в личных чатах туда попадает никнейм отправителя, как в ДЗ_3)
- текст сообщения (его можно немного посчистить (***по желанию***) - например, удалить из него тэги)
- дата
- (***по желанию***) реакции

Примерный результат выполнения всех заданий вы можете найти в той же [папке](https://disk.yandex.ru/d/druM43Rd5bdiQg)

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import codecs
import re

In [ ]:
f = codecs.open("/content/Системный Блокъ.html", 'r', 'utf-8') # читаем код html из файла
telegram = f.read() # записываем его в переменную
f.close()

soup = BeautifulSoup(telegram)
soup

Советы: можно идти по тегам в супе, а можно использовать регулярные выражения к содержимому тэга, оставлю одну из возможных подсказок (но разбор тэгов супом предпочтительнее, задачу можно также решить любым другим способом)

In [ ]:
names = []
texts = []
dates = []
reactions = []

for i in soup.find_all('div', {'class' : "message default clearfix"}):
  #названия
  names.append(i.find('div', {'class' : "from_name"}).text.strip())
  #тексты
  try:
    texts.append(i.find('div', {'class' : "text"}).text.strip())
  except:
    #print(i)
    texts.append(i.find('div', {'class' : "question bold"}).text.strip()) #в одном посте, как оказалось, нет текста, только опрос - поэтому в этом случае вместо текста отправила текст опроса
  #даты
  date = i.find('div', {'class' : "pull_right date details"}) #даты спрятаны в атрибут "title" внутри div
  dates.append(date['title'].strip())
  #реакции
  try:
    reactions.append(i.find('div', {'class' : "reactions"}).text.strip())
  except:
    reactions.append('None') #видимо не везде есть реакции, потому что без try-except выдает ошибку

In [ ]:
#реакции некрасиво отображаются с кучей пробелов и переносов, можно почистить
for post in range(len(reactions)):
  reactions[post] = re.sub(r'\n', '', reactions[post])

In [ ]:
print(len(names) == len(texts) == len(dates) == len(reactions)) #проверяем, что ничего не потерялось

True


In [ ]:
df = pd.DataFrame([names, texts, dates, reactions])
df = df.T
df.columns = ['name', 'text', 'date', 'reactions']
df

,name,text,date,reactions
0,Системный Блокъ,Что происходит в интернете: Оксана Мороз о циф...,03.01.2024 19:58:07,❤ 31 🎄 5 👍 ...
1,Системный Блокъ,"Танцы, эрос и зачатие: о чем писали «Платоновс...",06.01.2024 21:02:31,👍 14 🔥 11 ❤ ...
2,Системный Блокъ,Читаем секретные письма опальной королевы: ист...,10.01.2024 22:27:56,🔥 15 ❤ 12 👍 ...
3,Системный Блокъ,От Эдисона до Spotify: история форматов музыки...,11.01.2024 19:57:30,🔥 20 👍 16 ❤ ...
4,Системный Блокъ,"Больше, чем энциклопедия: Википедии 23 года!По...",15.01.2024 21:01:54,🍾 17 ❤ 11 👍 ...
...,...,...,...,...
199,Системный Блокъ,"Утечка доступа к Sora, открытые аналоги OpenAI...",02.12.2024 20:03:01,👍 15 🔥 9 ❤ ...
200,Системный Блокъ,Ресурсы для цифровых стиховедов: поэтические к...,05.12.2024 20:02:08,❤ 23 👍 15 🔥 ...
201,Системный Блокъ,"Позвони мне, позвони: как мобильные данные пом...",08.12.2024 20:10:36,🔥 23 ❤ 10 👍 ...
202,Системный Блокъ,ChatGPT — 2 года!Два года назад состоялся рели...,09.12.2024 21:36:03,❤ 13 🔥 9 👍 ...


## Часть 2

Парсинг сайта, на выбор:
- [Детские вопросы](https://elementy.ru/email) в журнале "Элементы" (немного легче)
- [Рубрика "Ускользающий мир"](https://www.nkj.ru/special/mir/) журнала "Наука и жизнь"

Общий алгоритм:
- смотрим на 1 и последнюю страницу и с помощью range(от, до+1) генерируем ссылки на страницы
- проходим по каждой странице и парсим ссылки на новости (важно собрать нужные и очистить от лишних!). На момент начала задания было 88 Детских вопроса и 213 текстов Ускользающего мира (ближе к дедлайну их может стать немного больше)
- переходим к каждой новости и парсим из нее:
  - заголовок, автор ответа, текст, ссылку для Детских вопросов
  - заголовок, дату, текст и ссылку для Ускользающего мира (***опционально*** - также автора, источник, тэги, но учтите, что они есть не в каждой статье)
- сохраняем результаты в датафрейм, не забудьте озаглавить столбцы

Поскольку мы парсим реальные сайты, вежливый парсинг с time и задержкой в несколько секунд будет плюсом!

In [ ]:
import time

In [ ]:
#код выполняется 8 минут
links = []
titles = []
dates = []
texts = []
for i in range(1,12): #на сайте 11 страниц
  page = requests.get('https://www.nkj.ru/special/mir/?PAGEN_1=' + str(i))
  time.sleep(1)
  page.encoding = 'utf-8'
  soup = BeautifulSoup(page.text, features = 'html.parser')
  for article in soup.find_all('article'): #все статьи – под тегом "article"
    for i in article.find_all('h2'): #в h2 хранятся заголовки и ссылки
      a = i.find('a')
      links.append('https://nkj.ru' + a.get("href"))
    titles.append(article.find('h2').text.strip())
    dates.append(article.find('time').text.strip())
for article in links: #отдельно проходимся по ссылке для каждой статьи
  page_article = requests.get(article)
  time.sleep(1)
  souparticle = BeautifulSoup(page_article.text, features = 'html.parser')
  text_list = [p.text for p in souparticle.find('article').find_all('p') if p.get('style') == None] #пытаюсь удалить дату, которая всегда есть в начале текста. сам текст в нескольких абзацах, каждый из них - в отдельном тэге p
  texts.append(' '.join(text_list)) #соединяем все абзацы, добавляем в список текстов



In [ ]:
#в текстах очень много лишних переносов
for text in range(len(texts)):
  texts[text] = re.sub('\n', '', texts[text])

In [ ]:
df_website = pd.DataFrame([links, titles, dates, texts])
df_website = df_website.T
df_website.columns = ['link', 'title', 'date', 'text']
df_website

,link,title,date,text
0,https://nkj.ru/special/mir/52496/,Откуда в озере Байкал взялась байкальская нерпа?,7 декабря 2024,"О единственном млекопитающем, живущем в водах ..."
1,https://nkj.ru/special/mir/52357/,"Чем больше аргали, тем лучше",25 ноября 2024,Впервые за историю 10-летнего мониторинга арга...
2,https://nkj.ru/special/mir/52327/,День моржа,25 ноября 2024,Сотрудники заповедных территорий рассказали об...
3,https://nkj.ru/special/mir/52270/,"Зажировка всем нужна, зажировка всем важна",12 ноября 2024,Обитатели заповедных территорий страны готовят...
4,https://nkj.ru/special/mir/52265/,«Синица к избе – зима на дворе»: как на запове...,12 ноября 2024,12 ноября отмечается Синичкин день. Экологичес...
...,...,...,...,...
208,https://nkj.ru/special/mir/38811/,Наука в режиме самоизоляции,10 апреля 2020,Как пандемия коронавируса повлияла на работу с...
209,https://nkj.ru/special/mir/38510/,Онлайн-встреча с участниками экспедиции «По сл...,3 апреля 2020,3 апреля в 12 часов по мск состоится онлайн-вс...
210,https://nkj.ru/special/mir/38431/,В объективе – снежный барс!,29 марта 2020,На Алтае впервые в России сфотографировали ирб...
211,https://nkj.ru/special/mir/38394/,Стартовала экспедиция по мониторингу снежного ...,21 марта 2020,В национальном парке «Сайлюгемский» начался вт...


In [ ]:
#сохраняем датафреймы
df.to_csv('telegram.csv')
df_website.to_csv('website.csv')